In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.metrics import roc_curve, auc, log_loss
from sklearn.metrics import classification_report, precision_recall_curve, auc
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm

import torch 
import os
import torch

import warnings
warnings.filterwarnings("ignore")

In [3]:
is_cuda = torch.cuda.is_available()
device = torch.device('cuda' if is_cuda else 'cpu')

print(device)

cuda


In [4]:
## Load original data
data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/return_feature_train.csv"
data = pd.read_csv(data_path, low_memory=False)

##load test data
test_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/return_feature_test.csv"
test_data = pd.read_csv(test_data_path)

In [5]:
keep_features = ['grade', 'term_months', 'total_pymnt', 'total_pymnt_inv','total_il_high_credit_limit', 'loan_amnt']

data_classification = data.copy()
test_classification = test_data.copy()

data_classification = data_classification.drop(columns = keep_features)
test_classification = test_classification.drop(columns = keep_features)

In [6]:
data_classification.head()

,last_fico_range_high,mo_sin_old_rev_tl_op,int_rate,annual_inc,dti,installment,zip_code,revol_util,fico_range_low,total_rev_hi_lim,revol_bal,bc_util,avg_cur_bal,credit_history_years,loan_status
0,744.0,126.0,0.0917,50000.0,4.66,140.27,622.0,0.768,710.0,7700.0,5913.0,76.8,64898.0,11,0
1,739.0,151.0,0.1249,60000.0,16.14,568.64,800.0,0.865,685.0,27700.0,23949.0,87.1,1831.0,12,0
2,614.0,292.0,0.0819,75000.0,24.67,244.41,350.0,0.858,680.0,20300.0,17425.0,96.3,19333.0,24,0
3,604.0,206.0,0.2449,67500.0,15.81,492.45,301.0,0.396,675.0,53300.0,21088.0,39.4,6661.0,17,1
4,654.0,424.0,0.1199,70000.0,23.70,332.10,770.0,0.847,660.0,22725.0,19249.0,94.9,25393.0,36,0


In [7]:
data_x = data_classification.drop(columns='loan_status')
data_y = data_classification[['loan_status']]

X_train, X_val, y_train, y_val = train_test_split(data_x, data_y, train_size=0.8, test_size=0.2, random_state=42, stratify=data_y)

In [8]:
train_classification = pd.concat([X_train, y_train], axis=1)

## CTGAN 적용

In [9]:
label1_data_path = "C:/Users/GCU/Lending_club/Data_Analysis_lending-club/Dataset/return_feature_train_label_1.csv"
label1_data = pd.read_csv(label1_data_path, low_memory=False)

In [11]:
label1_data.columns

Index(['last_fico_range_high', 'mo_sin_old_rev_tl_op', 'int_rate',
       'annual_inc', 'dti', 'installment', 'zip_code', 'revol_util',
       'fico_range_low', 'total_rev_hi_lim', 'revol_bal', 'bc_util',
       'avg_cur_bal', 'total_il_high_credit_limit', 'grade', 'total_pymnt',
       'total_pymnt_inv', 'loan_amnt', 'credit_history_years', 'term_months',
       'loan_status'],
      dtype='object')

In [ ]:
import pandas as pd
from sdv.metadata import SingleTableMetadata
from sdv.single_table import CTGANSynthesizer
import torch
import os


N_CLS_PER_GEN = 550_000
OUTPUT_PATH = 'synthetic_ctgan_data_class1.csv'


discrete_columns = ['zip_code', 'grade', 'term_months', 'loan_status']

metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=label1_data)

for column in discrete_columns:
    metadata.update_column(column_name=column, sdtype='categorical')

synthesizer = CTGANSynthesizer(
    metadata,
    epochs=500,
    cuda=torch.cuda.is_available(),  # CUDA 사용 여부
    verbose=True
)

print("🧪 Start training CTGAN on class 1 data...")
synthesizer.fit(label1_data)
print("✅ Training completed.")

print(f"🔄 Sampling {N_CLS_PER_GEN} synthetic samples...")
synthetic_data = synthesizer.sample(num_rows=N_CLS_PER_GEN)

os.makedirs("ctgan_output", exist_ok=True)
output_file = os.path.join("ctgan_output", OUTPUT_PATH)
synthetic_data.to_csv(output_file, index=False)
print(f"✅ Synthetic data saved to: {output_file}")

🧪 Start training CTGAN on class 1 data...
PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name       Est # of Columns (CTGAN)
last_fico_range_high       11
mo_sin_old_rev_tl_op       11
int_rate                   11
annual_inc                 11
dti                        11
installment                11
zip_code                   890
revol_util                 11
fico_range_low             11
total_rev_hi_lim           11
revol_bal                  11
bc_util                    11
avg_cur_bal                11
total_il_high_credit_limit 11
grade                      7
total_pymnt                11
total_pymnt_inv            11
loan_amnt                  11
credit_history_years       11
term_months                2
loan_status                1

We recommend preprocessing discrete columns that can have many values, using 'update_transformers'. Or you may drop columns that ar